In [ ]:
from datetime import datetime
import requests

head = {
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/87.0.4280.88 Safari/537.36 "
}


class Intra_Day:
    baseNumber = None
    session = None
    ticker = None

    def __init__(self, ticker):
        self.baseNumber = 0
        self.session = requests.session()
        self.ticker = ticker
        self.session.get("https://www.nseindia.com", headers=head)
        self.session.get("https://www.nseindia.com/get-quotes/equity?symbol=" + ticker, headers=head)

    def secondsTotime(self, seconds):
        seconds = seconds % (24 * 3600)
        hour = seconds // 3600
        seconds %= 3600
        minutes = seconds // 60
        seconds %= 60
        return hour, minutes, seconds

    def dateCalculator(self, num):
        if self.baseNumber == 0:
            self.baseNumber = (num // 100000) * 100000
        num = abs(num - self.baseNumber)
        num = num / 1000
        num = int(num)
        today = datetime.today()
        (h, m, s) = self.secondsTotime(num)
        return datetime(today.year, today.month, today.day, 9 + h, m, s)

    def intraDay(self,):
        preopen_url = "https://www.nseindia.com/api/chart-databyindex?index=" + self.ticker + "&preopen=true"
        open_url = "https://www.nseindia.com/api/chart-databyindex?index=" + self.ticker
        preopen_webdata = self.session.get(url=preopen_url, headers=head)
        opened_webdata = self.session.get(url=open_url, headers=head)
        timestamp = []
        data = []

        for (i, j) in preopen_webdata.json()['grapthData']:
            timestamp.append(self.dateCalculator(i))
            data.append(j)

        for (i, j) in opened_webdata.json()['grapthData']:
            timestamp.append(self.dateCalculator(i))
            data.append(j)
        return timestamp, data

    def nifty_intraDay(self):
        varient = self.ticker
        varient = varient.upper()
        varient = varient.replace(' ', '%20')
        varient = varient.replace('-', '%20')
        preopen_url = "https://www.nseindia.com/api/chart-databyindex?index={}&indices=true&preopen=true".format(
            varient)
        open_url = "https://www.nseindia.com/api/chart-databyindex?index={}&indices=true".format(varient)

        preopen_webdata = self.session.get(url=preopen_url, headers=head)
        open_webdata = self.session.get(url=open_url, headers=head)
        data = []
        timestamp = []

        for (i, j) in preopen_webdata.json()['grapthData']:
            data.append(j)
            timestamp.append(self.dateCalculator(i))

        for (i, j) in open_webdata.json()['grapthData']:
            data.append(j)
            timestamp.append(self.dateCalculator(i))

        return timestamp, data


In [ ]:
ID = Intra_Day('NIFTY 50')
timeStamp, dataPoints = ID.nifty_intraDay()

In [10]:
dataPoints

[24892.85,
 24883.65,
 24874.6,
 24880.95,
 24881.2,
 24857.65,
 24852.25,
 24855.15,
 24851.05,
 24851.25,
 24854.55,
 24854.95,
 24852.95,
 24855.1,
 24851.9,
 24848.6,
 24847.6,
 24846.75,
 24847.7,
 24848.45,
 24843.65,
 24840.25,
 24842.95,
 24842.1,
 24842.1,
 24843.1,
 24840.8,
 24841.7,
 24836.45,
 24834.55,
 24839.1,
 24835,
 24834.7,
 24834.6,
 24782.85,
 24782.85,
 24782.55,
 24782.25,
 24791.65,
 24791.15,
 24789.65,
 24789.7,
 24782.65,
 24781.75,
 24783.55,
 24785.35,
 24783.4,
 24782.9,
 24784.8,
 24784.75,
 24784.4,
 24783.8,
 24786,
 24789.5,
 24788.35,
 24788.35,
 24788.1,
 24778,
 24779.05,
 24784.95,
 24793.85,
 24769.2,
 24795.25,
 24769.7,
 24761.25,
 24761.9,
 24769.95,
 24773.8,
 24776.1,
 24805.1,
 24800.75,
 24797.85,
 24792.25,
 24793.5,
 24793.4,
 24798.8,
 24797.6,
 24797.3,
 24797.7,
 24797.7,
 24797.8,
 24797.95,
 24798.05,
 24797.15,
 24797.3,
 24797.75,
 24798.05,
 24798.2,
 24798.35,
 24798.75,
 24800.55,
 24803.25,
 24803.4,
 24803.4,
 24803.7,
 24803

In [ ]:
import requests
import pandas as pd
import io

In [ ]:
headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
                            'AppleWebKit/537.36 '
                            '(KHTML, like Gecko) '
                            'Chrome/80.0.3987.149 Safari/537.36',
            'accept-language': 'en,gu;q=0.9,hi;q=0.8', 
            'accept-encoding': 'gzip, deflate, br'}

def get_nse_payload(url: str) -> requests.models.Response:

    """
    Returns response

    url: (samples)
    * https://nsearchives.nseindia.com/content/nsccl/C_CATG.T01 for margin groups

    Response processing examples:
    ---
    `df = pd.read_csv(io.StringIO(response.text))` for csv type response output 

    """

    base_url = 'https://www.nseindia.com'
    
    with requests.Session() as session:
        request = session.get(base_url, headers=headers, timeout=5)
        cookies = dict(request.cookies)
        response = session.get(url, headers=headers, timeout=5, cookies=cookies)

    return response


def get_raw_nse_margin_df() -> pd.DataFrame:
    """Returns raw df of margins from nse website"""

    base_url = 'https://nsearchives.nseindia.com/archives/nsccl/var/C_VAR1_'
    date_url = '22042024' # Loop through formatted_dates
    hyphen = '_'
    serial = str(1) # loop through `reversed(range(1, 7))`
    end_url = '.DAT'

    date_range = pd.date_range(end=pd.Timestamp.today()+pd.Timedelta(days=3), periods=7)
    formatted_dates = [date.strftime("%d%m%Y") for date in reversed(date_range)]

    serial_range = [i for i in reversed(range(1, 7))]

    for date in formatted_dates:
        date_url = date
        for i in serial_range:
            serial = str(i)
            url = base_url+date_url+hyphen+serial+end_url
            
            response = get_nse_payload(url)
            df = get_parsed_response_df(response)
            if not df.empty:
                # print(url)
                # print(f"got df! {df.head(1)}")
                return df

def get_parsed_response_df(response: requests.models.Response) -> pd.DataFrame:
    """Parses responses from `get_nse_payload` and returns df"""

    try:
        out = pd.read_csv(io.StringIO(response.text))
    except pd.errors.ParserError:
        out = pd.DataFrame([])

    return out

In [ ]:
get_raw_nse_margin_df()
